In [1]:
import os
#set your environment variable for SSL certificate
certi_path = r"C:/laragon/bin/python/python-3.10/Lib/site-packages/pip/_vendor/certifi/cacert.pem"
os.environ['REQUESTS_CA_BUNDLE'] = certi_path
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import random
import time
import json
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [2]:
def initialize_driver(headers_list, proxy_list):
    options = Options()
    #select a random user-agent from the list
    user_agent = random.choice(headers_list)["User-Agent"]
    options.add_argument(f"user-agent={user_agent}")
    
    #select a random proxy from the list
    proxy = random.choice(proxy_list)
    if proxy:
        options.add_argument(f"--proxy-server={proxy}")
    
    #add some common options
    #options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--ignore-certificate-errors")

    #initialize Chrome WebDriver with the specified options
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    #set implicit wait of 10sec
    driver.implicitly_wait(10)

    return driver

In [3]:
def MainPage(driver, url):
    driver.get(url)
    time.sleep(3)

In [4]:
def nbHistoirs(driver):
    try:
        liste_histoires = driver.find_element(By.ID, "liste_histoires")
        search_results = driver.find_elements(By.TAG_NAME, "article")
        search_results_count = len(search_results)
        return search_results_count

    except Exception as e:
        print("An error occurred in NB_OFFER:", str(e))
        return 0

In [5]:
def nbHisytoryTot(driver):
    try:
        search_results = driver.find_element(By.CSS_SELECTOR, 'div[class="row align-items-center gy-2 mb-4 pb-1 pb-sm-2 pb-lg-3"]')
        text_nb_hostory_tot = search_results.find_element(By.CSS_SELECTOR, 'h2[class="mb-lg-0"]').text.split('(')[1].split(')')[0]
        return int(text_nb_hostory_tot)

    except Exception as e:
        print("An error occurred in NB_OFFER:", str(e))
        return 0  # Or handle the exception as needed

In [6]:
def Click(driver, pos):
    '''Click on the link'''
    try:
        element = driver.find_elements(By.CSS_SELECTOR, 'a[class="btn btn-sm btn-primary mb-1"]')[pos]
        element.click()
        return 1

    except Exception as e:
        print("An error occurred in CLICK:", str(e))
        return 0

In [7]:
def ClickMoreHistory(driver):
    '''Click on the link'''
    try:
        element = driver.find_element(By.ID, "loadmorebtn")
        element.click()
        return 1

    except Exception as e:
        print("An error occurred in CLICK:", str(e))
        return 0

In [10]:
def GetDatas(driver, story):
    time.sleep(3)
    try:
        age = driver.find_elements(By.CSS_SELECTOR, 'span[class="badge border border-light text-light fs-sm mb-1"]')[1].text
        title = driver.find_element(By.CSS_SELECTOR, 'h1[class="display-2 mb-4"]').text
        lien_type_history = driver.find_element(By.CSS_SELECTOR, 'ol[class="pt-lg-3 pb-lg-4 pb-2 breadcrumb"]').find_elements(By.TAG_NAME, 'li')
        category = lien_type_history[3].find_element(By.TAG_NAME, 'a').text
        genre = lien_type_history[4].find_element(By.TAG_NAME, 'a').text

        current_story = [age, title, history, category, genre]
        story.append(current_story)

        # Écriture dans le fichier
        with open('history.txt', 'a') as f:
            f.write(str(current_story))
            print(f'Écriture du travail : {driver.current_url}')
            f.write('\n')

        return 1

    except Exception as e:
        print(f"Erreur lors de l'analyse HTML : {e}")


In [11]:
driver = initialize_driver([
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Ch-Ua": "\"Chromium\";v=\"92\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"92\"",
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15"
    }
], [False])

In [12]:
MainPage(driver, r"https://www.meshistoiresdusoir.fr/g/contes/")


In [13]:
nbHisytoryTotVar = nbHisytoryTot(driver)

In [19]:
nbHistoryByPageVar = nbHistoirs(driver)


In [26]:
nb_page = nbHisytoryTotVar // nbHistoryByPageVar
if nbHisytoryTotVar % nbHistoryByPageVar != 0:
    nb_page = nb_page + 1


In [28]:
story = []
nb_history_page_init = nbHistoirs(driver)
for a in range(0, nb_page - 1):
    nb_history_page = nbHistoirs(driver)
    for i in range((a * nb_history_page_init), nb_history_page):
        Click(driver, i)
        GetDatas(driver, story)
        driver.back()
    time.sleep(3)
    ClickMoreHistory(driver)
    time.sleep(3)

An error occurred in NB_OFFER: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="liste_histoires"]"}
  (Session info: chrome=122.0.6261.131); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x011E8D03+51395]
	(No symbol) [0x01155F61]
	(No symbol) [0x0100E13A]
	(No symbol) [0x010462BB]
	(No symbol) [0x010463EB]
	(No symbol) [0x0107C162]
	(No symbol) [0x01063ED4]
	(No symbol) [0x0107A570]
	(No symbol) [0x01063C26]
	(No symbol) [0x0103C629]
	(No symbol) [0x0103D40D]
	GetHandleVerifier [0x015668D3+3712147]
	GetHandleVerifier [0x015A5CBA+3971194]
	GetHandleVerifier [0x015A0FA8+3951464]
	GetHandleVerifier [0x01299D09+776393]
	(No symbol) [0x01161734]
	(No symbol) [0x0115C618]
	(No symbol) [0x0115C7C9]
	(No symbol) [0x0114DDF0]
	BaseThreadInitThunk [0x76B67BA9+25]
	RtlInitializeExceptionChain [0x76EFBDAB+107]
	RtlClearBits

KeyboardInterrupt: 